In [87]:
#%pip install openpyxl

In [88]:
import pandas as pd
import json
import os
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import locale

In [89]:
# Read Transaction Excel data into a Pandas dataframe
file = r'C:\Users\admin\Documents\GitHub\davemccallme.github.io\scratch\sunbursts graphs2\Trxn-Data-BarGraphs.xlsx'
df = pd.read_excel(file)
df['Value'] = pd.to_numeric(df['Value'])
print(df.head())



          Root Level               Level 1               Level 2    Value
0  Call Center - IVR              Payments              Payments  4474000
1  Call Center - IVR               Billing               Billing  1241000
2  Call Center - IVR             Pay Plans             Pay Plans   209000
3  Call Center - IVR       Account Balance       Account Balance   205000
4  Call Center - IVR  Start/Stop/ Transfer  Start/Stop/ Transfer   337000


In [90]:
# Convert dataframe to a hierarchical dictionary
def make_hierarchy_dict(df, path=[], value_col="Value", threshold=10):
    if len(path) == df.columns.nlevels - 1:
        total = int(df[value_col].sum())
        if total < threshold:
            return None
        else:
            return total
    else:
        level = df.columns[len(path)]
        subdict = {}
        for key in df[level].dropna().unique():
            subdf = df[df[level] == key].drop(columns=[level])
            subhierarchy_dict = make_hierarchy_dict(subdf, path + [key], value_col, threshold)
            if subhierarchy_dict is not None:
                subdict[key] = subhierarchy_dict
        if len(subdict) == 0:
            return None
        else:
            return subdict

hierarchy_dict = make_hierarchy_dict(df.set_index(list(df.columns[:3])), value_col="Value", threshold=10)

print(hierarchy_dict)

106324215


In [91]:
# Convert hierarchical dictionary to Plotly-compatible JSON
def make_plotly_sunburst(hierarchy_dict, name=""):
    if isinstance(hierarchy_dict, dict):
        children = []
        for key, value in hierarchy_dict.items():
            children.append(make_plotly_sunburst(value, key))
        return {
            "name": name,
            "children": children
        }
    else:
        return {
            "label": name,
            "value": hierarchy_dict
        }

In [92]:
#Error Check rows for data discrepencies
for i, row in df.iterrows():
    try:
        fig = px.sunburst(df.loc[[i]], path=["Root Level", "Level 1", "Level 2"], values='Value')
    except ValueError as e:
        print(e)

In [93]:
# Group the data by Root Level and sum the values
grouped_data = df.groupby(['Root Level'])['Value'].sum().reset_index()

# Sort the grouped_data by Value column in descending order
grouped_data = grouped_data.sort_values(by='Value', ascending=False)

# Create a dictionary to map the Root Level to its corresponding color
color_map = {'Web': 'orange', 'Other Electronic': 'gray', 'Mail': 'burlywood', 'Call Center - IVR': 'deepskyblue','Call Center - Other':'lightblue', \
              'Call Center - Live Agent':'Turquoise', 'NPC': 'gold'}

# Create the bar chart trace
bar_trace = go.Bar(
    x=grouped_data['Root Level'],
    y=grouped_data['Value'],
    marker=dict(color=[color_map.get(root_level, 'grey') for root_level in grouped_data['Root Level']]),
    text=[f"{value:,}" for value in grouped_data['Value']],
    textposition='auto'
)

# Create the bar chart layout
bar_layout = go.Layout(
    title='Total Customer Interactions by Transaction Channel',
    xaxis=dict(title='Channel'),
    yaxis=dict(title='Total Interactions'),
    legend=dict(title=dict(text='Transaction Channels'))
)

# Create the bar chart figure
bar_fig_root = go.Figure(data=[bar_trace], layout=bar_layout)

# Display the bar chart
bar_fig_root.show()


In [94]:
# Export the chart as an HTML file
pyo.plot(bar_fig_root, filename='Trxn_root_barchart.html', auto_open=True)

'Trxn_root_barchart.html'

In [100]:
# Set the locale to use comma separators for thousands
locale.setlocale(locale.LC_ALL, '')

# Group the data by Root Level and Level 1 and sum the values
grouped_data = df.groupby(['Root Level', 'Level 1'])['Value'].sum().reset_index()

# Sort the grouped_data by Value column in descending order
grouped_data = grouped_data.sort_values(by='Value', ascending=False)

# Create a dictionary to map the Root Level to its corresponding color
color_map = {'Web': 'orange', 'Other Electronic': 'gray', 'Mail': 'burlywood',   'Call Center - Live Agent': 'turquoise', 'Call Center - IVR': 'deepskyblue', 'Call Center - Other':'lightblue', 'NPC': 'gold'}


# Create a list to store the bar chart traces
bar_traces = []

# Loop through each Root Level and create a stacked bar chart trace for its corresponding Level 1 data
for root_level in grouped_data['Root Level'].unique():
    temp_df = grouped_data[grouped_data['Root Level'] == root_level]
    values = temp_df['Value'].values
    formatted_values = [locale.format_string("%d", val, grouping=True) for val in values] # Format values with comma separators
    bar_trace = go.Bar(
        x=temp_df['Level 1'],
        y=values,
        text=formatted_values, # Set the formatted values as the text labels
        textposition='auto', # Position the text labels automatically above the bars
        name=root_level,
        marker=dict(color=color_map[root_level])
    )
    bar_traces.append(bar_trace)

# Create the stacked bar chart layout
bar_layout = go.Layout(
    title='Total Customer Interactions by Service',
    xaxis=dict(title='Service'),
    yaxis=dict(title='Total Interactions'),
    barmode='stack',
    height=900,
    legend=dict(title=dict(text='Transaction Channels'))
)

# Create the stacked bar chart figure
bar_fig_root_L1 = go.Figure(data=bar_traces, layout=bar_layout)

# Display the stacked bar chart
bar_fig_root_L1.show()


In [96]:
# Export the chart as an HTML file
pyo.plot(bar_fig_root_L1, filename='Trxn_root_L1_barchart.html', auto_open=True)

'Trxn_root_L1_barchart.html'

In [99]:
# Set the locale to use comma separators for thousands
locale.setlocale(locale.LC_ALL, '')

# Group the data by Root Level and Level 2 and sum the values
grouped_data = df.groupby(['Root Level', 'Level 2'])['Value'].sum().reset_index()

# Sort the grouped_data by Value column in descending order
grouped_data = grouped_data.sort_values(by='Value', ascending=False) # Sort by descending order

# Create a dictionary to map the Root Level to its corresponding color
color_map = {'Web': 'orange', 'Other Electronic': 'gray', 'Mail': 'burlywood',   'Call Center - Live Agent': 'turquoise', 'Call Center - IVR': 'deepskyblue', 'Call Center - Other':'lightblue', 'NPC': 'gold'}



# Create a list to store the bar chart traces
bar_traces = []

# Loop through each Root Level and create a stacked bar chart trace for its corresponding Level 2 data
for root_level in grouped_data['Root Level'].unique():
    temp_df = grouped_data[grouped_data['Root Level'] == root_level]
    values = temp_df['Value'].values
    formatted_values = [locale.format_string("%d", val, grouping=True) for val in values] # Format values with comma separators
    y_values = temp_df['Level 2'].values
    bar_trace = go.Bar(
        y=y_values,
        x=values,
        text=formatted_values,
        textposition='auto',
        name=root_level,
        orientation='h',
        marker=dict(color=color_map[root_level])
    )
    bar_traces.append(bar_trace)

# Create the stacked bar chart layout
bar_layout = go.Layout(
    title='Total Customer Interactions by Service - Breakdown',
    xaxis=dict(title='Total Interactions'),
    yaxis=dict(title='Service Detail (from lowest to highest)'),
    barmode='stack',
    height=1600,
    legend=dict(title=dict(text='Transaction Channels'))
)

# Create the stacked bar chart figure
bar_fig_L2 = go.Figure(data=bar_traces, layout=bar_layout)

# Display the stacked bar chart
bar_fig_L2.show()


In [98]:
# Export the chart as an HTML file
pyo.plot(bar_fig_L2, filename='Trxn_barchart_L2.html', auto_open=True)

'Trxn_barchart_L2.html'